In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix,precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix
def get_clf_eval(y_test,pred=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test,pred)
    f1=f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}\
    F1: {3:.4f}'.format(accuracy,precision,recall,f1))
def get_outlier(df=None, column=None, weight=1.5):
    fraud=df[column]
    quantile_25=np.percentile(fraud.values,25)
    quantile_75=np.percentile(fraud.values,75)
    iqr=quantile_75-quantile_25
    iqr_weight=iqr * weight
    lowest_val=quantile_25-iqr_weight
    highest_val=quantile_75 + iqr_weight
    outlier_index= fraud[(fraud<lowest_val) | (fraud>highest_val)].index
   # df.drop(outlier_index,axis=0,inplace=True)
    return outlier_index

In [152]:
vote_df=pd.read_csv('PR2_train.csv')
vote_df

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,43375,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,...,1,1,1,1,1,0,1,0,1,1
1,4486,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,...,1,1,1,1,1,0,1,0,1,1
2,10262,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,...,1,1,0,1,1,0,1,0,1,0
3,14088,4.0,182,1.0,2969,1.0,1955,4.0,4630,1.0,...,0,1,0,1,1,0,1,0,1,1
4,13398,4.0,549,5.0,1679,5.0,481,3.0,595,1.0,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,16511,1.0,312,5.0,965,5.0,499,1.0,1188,1.0,...,1,1,0,1,1,0,1,1,1,1
36421,33507,3.0,474,4.0,1033,5.0,602,2.0,703,1.0,...,1,1,1,1,1,0,1,1,1,1
36422,44917,2.0,617,4.0,1262,2.0,709,1.0,1063,2.0,...,1,1,0,1,1,0,1,1,1,1
36423,36126,5.0,294,1.0,985,1.0,504,3.0,613,5.0,...,1,1,1,1,1,1,1,1,1,1


In [153]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le = le.fit(vote_df['gender'])
vote_df['gender'] = le.transform(vote_df['gender'])
vote_df['gender']
le = preprocessing.LabelEncoder()
le = le.fit(vote_df['religion'])
vote_df['religion'] = le.transform(vote_df['religion'])
vote_df['religion']
le = preprocessing.LabelEncoder()
le = le.fit(vote_df['race'])
vote_df['race'] = le.transform(vote_df['race'])
vote_df['race']
le = preprocessing.LabelEncoder()
le = le.fit(vote_df['age_group'])
vote_df['age_group'] = le.transform(vote_df['age_group'])
vote_df['age_group']

vote_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36425 entries, 0 to 36424
Data columns (total 78 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       36425 non-null  int64  
 1   QaA         36425 non-null  float64
 2   QaE         36425 non-null  int64  
 3   QbA         36425 non-null  float64
 4   QbE         36425 non-null  int64  
 5   QcA         36425 non-null  float64
 6   QcE         36425 non-null  int64  
 7   QdA         36425 non-null  float64
 8   QdE         36425 non-null  int64  
 9   QeA         36425 non-null  float64
 10  QeE         36425 non-null  int64  
 11  QfA         36425 non-null  float64
 12  QfE         36425 non-null  int64  
 13  QgA         36425 non-null  float64
 14  QgE         36425 non-null  int64  
 15  QhA         36425 non-null  float64
 16  QhE         36425 non-null  int64  
 17  QiA         36425 non-null  float64
 18  QiE         36425 non-null  int64  
 19  QjA         36425 non-nul

In [154]:
x=vote_df.values.astype(float)
min_max_scaler=preprocessing.MinMaxScaler()
x_scaled=min_max_scaler.fit_transform(x)
vote_df=pd.DataFrame(x_scaled,columns=vote_df.columns)
vote_df

,index,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0.952648,0.00,0.000124,1.00,0.000322,1.00,0.000383,0.00,0.000440,0.00,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
1,0.098526,0.00,0.000211,1.00,0.000473,1.00,0.002294,0.00,0.000690,0.00,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0.225385,1.00,0.000242,0.00,0.000224,0.25,0.000687,0.00,0.001164,0.50,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.309416,0.75,0.000065,0.00,0.000911,0.00,0.002214,0.75,0.002965,0.00,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.294261,0.75,0.000217,1.00,0.000512,1.00,0.000523,0.50,0.000366,0.00,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,0.362632,0.00,0.000119,1.00,0.000291,1.00,0.000544,0.00,0.000748,0.00,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36421,0.735916,0.50,0.000186,0.75,0.000312,1.00,0.000662,0.25,0.000436,0.00,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36422,0.986515,0.25,0.000245,0.75,0.000383,0.25,0.000785,0.00,0.000668,0.25,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36423,0.793437,1.00,0.000111,0.00,0.000297,0.00,0.000550,0.50,0.000378,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


투표 비율 살펴보기.

인덱스 항목 제거하고 , 투표 여부인 61번째 항목 따로 뽑기.

관련성이 없어보이는 것들을 제거 해보겠습니다.
#vote_drop=vote_df.drop('index',axis=1,inplace=True)
#vote_drop2=vote_df.drop('familysize',axis=1,inplace=True)
지울것 : hand married engnat.. 
무응답자들을 평균치로 바꾸면 좋을 듯


In [156]:
droplist=['hand','index','familysize']
vote_droplist=vote_df.drop(droplist,axis=1,inplace=True)

이상치 데이터들을 확인해보겠습니다. 


In [157]:
vote_data.columns


Index(['QaA', 'QaE', 'QbA', 'QbE', 'QcA', 'QcE', 'QdA', 'QdE', 'QeA', 'QeE',
       'QfA', 'QfE', 'QgA', 'QgE', 'QhA', 'QhE', 'QiA', 'QiE', 'QjA', 'QjE',
       'QkA', 'QkE', 'QlA', 'QlE', 'QmA', 'QmE', 'QnA', 'QnE', 'QoA', 'QoE',
       'QpA', 'QpE', 'QqA', 'QqE', 'QrA', 'QrE', 'QsA', 'QsE', 'QtA', 'QtE',
       'age_group', 'education', 'gender', 'race', 'religion', 'tp01', 'tp02',
       'tp03', 'tp04', 'tp05', 'tp06', 'tp07', 'tp08', 'tp09', 'tp10', 'urban',
       'wf_01', 'wf_02', 'wf_03', 'wr_01', 'wr_02', 'wr_03', 'wr_04', 'wr_05',
       'wr_06', 'wr_07', 'wr_08', 'wr_09', 'wr_10', 'wr_11', 'wr_12'],
      dtype='object')

In [158]:
for i in vote_data.columns :
    outlier_index=get_outlier(df=vote_df,column=i,weight=1.5)
    print(i," 의 이상치 인덱스 : ",outlier_index)


QaA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QaE  의 이상치 인덱스 :  Int64Index([    7,    21,    31,    50,    71,    77,    86,   102,   107,
              110,
            ...
            36314, 36326, 36337, 36350, 36363, 36370, 36373, 36381, 36395,
            36415],
           dtype='int64', length=2680)
QbA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QbE  의 이상치 인덱스 :  Int64Index([    7,    15,    21,    43,    50,    55,    87,    90,   124,
              141,
            ...
            36350, 36360, 36364, 36370, 36379, 36387, 36393, 36401, 36406,
            36415],
           dtype='int64', length=2953)
QcA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QcE  의 이상치 인덱스 :  Int64Index([    7,    35,    41,    49,    50,    87,    93,    96,   101,
              115,
            ...
            36337, 36357, 36358, 36363, 36365, 36367, 36370, 36375, 36404,
            36415],
           dtype='int64', length=2946)
QdA  의 이상치 인덱스 :  Int64Index([    3,    12,    30,    36,    38,    41

데이터 손실을 방지하기 위해 이상치 데이터들을 미리 써놓겠습니다.
QaA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QaE  의 이상치 인덱스 :  Int64Index([    7,    21,    31,    50,    71,    77,    86,   102,   107,
              110,
            ...
            36314, 36326, 36337, 36350, 36363, 36370, 36373, 36381, 36395,
            36415],
           dtype='int64', length=2680)
QbA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QbE  의 이상치 인덱스 :  Int64Index([    7,    15,    21,    43,    50,    55,    87,    90,   124,
              141,
            ...
            36350, 36360, 36364, 36370, 36379, 36387, 36393, 36401, 36406,
            36415],
           dtype='int64', length=2953)
QcA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QcE  의 이상치 인덱스 :  Int64Index([    7,    35,    41,    49,    50,    87,    93,    96,   101,
              115,
            ...
            36337, 36357, 36358, 36363, 36365, 36367, 36370, 36375, 36404,
            36415],
           dtype='int64', length=2944)
QdA  의 이상치 인덱스 :  Int64Index([    3,    12,    30,    36,    38,    41,    69,    79,   114,
              145,
            ...
            36334, 36340, 36352, 36358, 36378, 36379, 36388, 36400, 36409,
            36416],
           dtype='int64', length=3191)
QdE  의 이상치 인덱스 :  Int64Index([    3,     9,    27,    33,    35,    41,    49,    50,    66,
               69,
            ...
            36355, 36370, 36374, 36383, 36392, 36393, 36394, 36404, 36411,
            36415],
           dtype='int64', length=2907)
QeA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QeE  의 이상치 인덱스 :  Int64Index([    1,    15,    21,    41,    50,    55,    72,    77,    82,
               87,
            ...
            36316, 36325, 36326, 36334, 36350, 36355, 36361, 36364, 36411,
            36415],
           dtype='int64', length=3068)
QfA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QfE  의 이상치 인덱스 :  Int64Index([    1,     3,    11,    15,    25,    31,    33,    41,    44,
               50,
            ...
            36356, 36360, 36363, 36364, 36366, 36373, 36392, 36393, 36412,
            36415],
           dtype='int64', length=2939)
QgA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QgE  의 이상치 인덱스 :  Int64Index([   29,    41,    49,    50,    53,    58,    69,    77,    94,
              124,
            ...
            36325, 36326, 36358, 36360, 36381, 36388, 36393, 36407, 36415,
            36420],
           dtype='int64', length=2794)
QhA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QhE  의 이상치 인덱스 :  Int64Index([   42,    55,    59,    75,    79,    87,   103,   105,   110,
              161,
            ...
            36306, 36311, 36326, 36346, 36350, 36374, 36393, 36406, 36415,
            36422],
           dtype='int64', length=2721)
QiA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QiE  의 이상치 인덱스 :  Int64Index([    3,     5,    28,    41,    42,    49,    51,    53,    56,
               77,
            ...
            36269, 36278, 36290, 36320, 36334, 36350, 36364, 36393, 36402,
            36415],
           dtype='int64', length=2937)
QjA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QjE  의 이상치 인덱스 :  Int64Index([    3,     4,    11,    21,    31,    35,    50,    79,    93,
               94,
            ...
            36336, 36337, 36350, 36352, 36370, 36376, 36392, 36393, 36404,
            36415],
           dtype='int64', length=3314)
QkA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QkE  의 이상치 인덱스 :  Int64Index([    1,    29,    50,    77,    81,    87,    97,   103,   110,
              111,
            ...
            36237, 36244, 36261, 36274, 36278, 36291, 36296, 36325, 36350,
            36375],
           dtype='int64', length=2888)
QlA  의 이상치 인덱스 :  Int64Index([    2,     3,    13,    58,    61,    65,    80,    87,    88,
               99,
            ...
            36339, 36381, 36386, 36388, 36389, 36391, 36393, 36415, 36419,
            36421],
           dtype='int64', length=3452)
QlE  의 이상치 인덱스 :  Int64Index([   15,    31,    33,    41,    49,    50,    66,    69,    77,
               85,
            ...
            36267, 36278, 36325, 36350, 36358, 36381, 36387, 36391, 36393,
            36420],
           dtype='int64', length=2764)
QmA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QmE  의 이상치 인덱스 :  Int64Index([    0,     5,    11,    31,    41,    45,    60,    77,    87,
               94,
            ...
            36317, 36320, 36365, 36369, 36373, 36378, 36393, 36406, 36412,
            36415],
           dtype='int64', length=3119)
QnA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QnE  의 이상치 인덱스 :  Int64Index([    7,    11,    31,    35,    38,    49,    50,    51,    55,
               69,
            ...
            36358, 36361, 36364, 36387, 36393, 36396, 36409, 36414, 36415,
            36417],
           dtype='int64', length=2816)
QoA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QoE  의 이상치 인덱스 :  Int64Index([   11,    14,    21,    29,    31,    41,    50,    55,    58,
               71,
            ...
            36274, 36303, 36337, 36364, 36370, 36376, 36385, 36402, 36415,
            36422],
           dtype='int64', length=2884)
QpA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QpE  의 이상치 인덱스 :  Int64Index([    1,     5,    31,    50,    62,    77,    97,   110,   124,
              125,
            ...
            36334, 36336, 36350, 36355, 36366, 36373, 36374, 36393, 36394,
            36415],
           dtype='int64', length=2784)
QqA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QqE  의 이상치 인덱스 :  Int64Index([    1,    31,    38,    50,    58,    63,    73,    77,    81,
              107,
            ...
            36304, 36337, 36350, 36364, 36370, 36373, 36376, 36387, 36391,
            36415],
           dtype='int64', length=2711)
QrA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QrE  의 이상치 인덱스 :  Int64Index([    7,    21,    33,    41,    42,    49,    50,    55,    63,
               69,
            ...
            36301, 36303, 36324, 36325, 36326, 36336, 36355, 36388, 36391,
            36415],
           dtype='int64', length=3105)
QsA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QsE  의 이상치 인덱스 :  Int64Index([    1,    11,    41,    50,    55,    77,   124,   125,   131,
              139,
            ...
            36278, 36279, 36285, 36314, 36320, 36337, 36350, 36351, 36373,
            36415],
           dtype='int64', length=3028)
QtA  의 이상치 인덱스 :  Int64Index([], dtype='int64')
QtE  의 이상치 인덱스 :  Int64Index([    6,    12,    35,    42,    50,    53,    69,    77,    79,
               87,
            ...
            36344, 36346, 36352, 36355, 36364, 36367, 36376, 36393, 36395,
            36415],
           dtype='int64', length=3064)
age_group  의 이상치 인덱스 :  Int64Index([], dtype='int64')
education  의 이상치 인덱스 :  Int64Index([   25,    33,    70,    78,   139,   200,   306,   330,   470,
              529,
            ...
            35466, 35541, 35553, 35650, 35747, 35904, 35931, 36225, 36236,
            36411],
           dtype='int64', length=432)
gender  의 이상치 인덱스 :  Int64Index([], dtype='int64')
race  의 이상치 인덱스 :  Int64Index([    1,     7,    13,    32,    39,    41,    49,    50,    66,
               75,
            ...
            36367, 36383, 36390, 36393, 36397, 36399, 36402, 36412, 36415,
            36420],
           dtype='int64', length=7487)
religion  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp01  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp02  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp03  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp04  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp05  의 이상치 인덱스 :  Int64Index([  105,   128,   221,   226,   284,   626,   636,   693,   739,
              838,
            ...
            36009, 36085, 36132, 36135, 36144, 36217, 36218, 36233, 36351,
            36377],
           dtype='int64', length=623)
tp06  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp07  의 이상치 인덱스 :  Int64Index([  230,   284,   592,   626,   739,   842,   941,  1269,  1281,
             1291,
            ...
            35085, 35136, 35180, 35706, 35735, 35910, 36085, 36142, 36218,
            36377],
           dtype='int64', length=260)
tp08  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp09  의 이상치 인덱스 :  Int64Index([], dtype='int64')
tp10  의 이상치 인덱스 :  Int64Index([], dtype='int64')
urban  의 이상치 인덱스 :  Int64Index([  405,   430,   637,   741,   821,  1213,  1245,  1295,  1521,
             1548,
            ...
            34736, 34815, 35040, 35054, 35141, 35181, 35263, 35729, 35960,
            36380],
           dtype='int64', length=258)
wf_01  의 이상치 인덱스 :  Int64Index([    8,    13,    25,    51,    56,    70,    71,    87,    91,
               95,
            ...
            36333, 36334, 36347, 36359, 36363, 36364, 36365, 36395, 36403,
            36414],
           dtype='int64', length=2932)
wf_02  의 이상치 인덱스 :  Int64Index([    0,     8,    10,    13,    20,    40,    51,    56,    60,
               70,
            ...
            36395, 36396, 36413, 36414, 36418, 36419, 36420, 36421, 36422,
            36423],
           dtype='int64', length=5948)
wf_03  의 이상치 인덱스 :  Int64Index([    8,    19,    43,    52,    56,    60,    70,    71,    73,
               74,
            ...
            36347, 36348, 36351, 36364, 36368, 36369, 36385, 36387, 36420,
            36422],
           dtype='int64', length=4320)
wr_01  의 이상치 인덱스 :  Int64Index([], dtype='int64')
wr_02  의 이상치 인덱스 :  Int64Index([    6,     7,    29,    33,    39,    66,    68,    86,   119,
              123,
            ...
            36284, 36289, 36312, 36320, 36334, 36336, 36339, 36356, 36384,
            36408],
           dtype='int64', length=2387)
wr_03  의 이상치 인덱스 :  Int64Index([    1,     6,     7,    11,    12,    15,    19,    49,    50,
               57,
            ...
            36403, 36405, 36409, 36412, 36414, 36415, 36421, 36422, 36423,
            36424],
           dtype='int64', length=7603)
wr_04  의 이상치 인덱스 :  Int64Index([    3,     7,    23,    32,    33,    39,    47,    64,    66,
               86,
            ...
            36355, 36361, 36365, 36373, 36393, 36397, 36400, 36402, 36408,
            36415],
           dtype='int64', length=4782)
wr_05  의 이상치 인덱스 :  Int64Index([    7,    15,    32,    33,    36,    41,    50,    54,    64,
               66,
            ...
            36350, 36356, 36361, 36370, 36393, 36399, 36400, 36402, 36408,
            36415],
           dtype='int64', length=3822)
wr_06  의 이상치 인덱스 :  Int64Index([], dtype='int64')
wr_07  의 이상치 인덱스 :  Int64Index([    7,    33,    41,    50,    64,    66,    87,   105,   119,
              123,
            ...
            36289, 36312, 36320, 36325, 36336, 36339, 36355, 36400, 36408,
            36415],
           dtype='int64', length=2226)
wr_08  의 이상치 인덱스 :  Int64Index([    7,    33,    41,    64,    66,    80,    87,   123,   149,
              161,
            ...
            36215, 36233, 36253, 36278, 36312, 36320, 36339, 36354, 36393,
            36415],
           dtype='int64', length=1034)
wr_09  의 이상치 인덱스 :  Int64Index([    4,    11,    12,    15,    18,    27,    31,    37,    49,
               57,
            ...
            36384, 36386, 36387, 36398, 36409, 36413, 36415, 36419, 36423,
            36424],
           dtype='int64', length=7382)
wr_10  의 이상치 인덱스 :  Int64Index([    7,    33,    38,    41,    64,    74,    87,   105,   119,
              121,
            ...
            36224, 36253, 36271, 36278, 36289, 36312, 36320, 36339, 36400,
            36415],
           dtype='int64', length=1984)
wr_11  의 이상치 인덱스 :  Int64Index([], dtype='int64')
wr_12  의 이상치 인덱스 :  Int64Index([    7,    29,    33,    45,    50,    86,   119,   123,   124,
              130,
            ...
            36278, 36289, 36301, 36312, 36334, 36336, 36339, 36356, 36399,
            36400],
           dtype='int64', length=1911)

In [165]:
vote_df.drop(outlier_index,axis=0,inplace=True)

In [159]:
vote_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36425 entries, 0 to 36424
Data columns (total 75 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   QaA        36425 non-null  float64
 1   QaE        36425 non-null  float64
 2   QbA        36425 non-null  float64
 3   QbE        36425 non-null  float64
 4   QcA        36425 non-null  float64
 5   QcE        36425 non-null  float64
 6   QdA        36425 non-null  float64
 7   QdE        36425 non-null  float64
 8   QeA        36425 non-null  float64
 9   QeE        36425 non-null  float64
 10  QfA        36425 non-null  float64
 11  QfE        36425 non-null  float64
 12  QgA        36425 non-null  float64
 13  QgE        36425 non-null  float64
 14  QhA        36425 non-null  float64
 15  QhE        36425 non-null  float64
 16  QiA        36425 non-null  float64
 17  QiE        36425 non-null  float64
 18  QjA        36425 non-null  float64
 19  QjE        36425 non-null  float64
 20  QkA   

In [160]:
myindexlist=[]
for i in range(0,74):
    myindexlist.append(i)
del myindexlist[58]
myindexlist

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73]

In [166]:
vote_result=vote_df['voted']
vote_data=vote_df.iloc[:,myindexlist]


,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,1.0,324,5.0,1067,5.0,359,1.0,709,1.0,742,...,1,1,1,1,1,0,1,0,1,1
1,1.0,534,5.0,1555,5.0,2024,1.0,1097,1.0,3077,...,1,1,1,1,1,0,1,0,1,1
2,5.0,609,1.0,749,2.0,624,1.0,1833,3.0,1474,...,1,1,0,1,1,0,1,0,1,0
3,4.0,182,1.0,2969,1.0,1955,4.0,4630,1.0,1321,...,0,1,0,1,1,0,1,0,1,1
4,4.0,549,5.0,1679,5.0,481,3.0,595,1.0,1202,...,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,1.0,312,5.0,965,5.0,499,1.0,1188,1.0,437,...,1,1,0,1,1,0,1,1,1,1
36421,3.0,474,4.0,1033,5.0,602,2.0,703,1.0,782,...,1,1,1,1,1,0,1,1,1,1
36422,2.0,617,4.0,1262,2.0,709,1.0,1063,2.0,1728,...,1,1,0,1,1,0,1,1,1,1
36423,5.0,294,1.0,985,1.0,504,3.0,613,5.0,551,...,1,1,1,1,1,1,1,1,1,1


테스트 트레인 세트 분류하기.

In [167]:
X_train,X_test,y_train,y_test=train_test_split(vote_data, vote_result, test_size=0.2, random_state=0)
print("트레인 데이터 분포 : \n",y_train.value_counts())
print("\n테스트 데이터 분포 : \n",y_test.value_counts())


트레인 데이터 분포 : 
 0.0    14948
1.0    12663
Name: voted, dtype: int64

테스트 데이터 분포 : 
 0.0    3743
1.0    3160
Name: voted, dtype: int64


비슷하게 나누어진 것 확인.

In [169]:
dt_clf=DecisionTreeClassifier(random_state=0)
rf_clf=RandomForestClassifier(random_state=0)
lr_clf=LogisticRegression()

dt_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
lr_clf.fit(X_train,y_train)

dt_pred=dt_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
lr_pred=lr_clf.predict(X_test)





print("결정 트리")
get_clf_eval(y_test,dt_pred)
print("랜덤 포레스트")
get_clf_eval(y_test,rf_pred)
print("로지스틱")
get_clf_eval(y_test,lr_pred)



결정 트리
오차 행렬
[[2404 1339]
 [1376 1784]]
정확도: 0.6067, 정밀도: 0.5712, 재현율: 0.5646    F1: 0.5679
랜덤 포레스트
오차 행렬
[[2468 1275]
 [ 959 2201]]
정확도: 0.6764, 정밀도: 0.6332, 재현율: 0.6965    F1: 0.6634
로지스틱
오차 행렬
[[2707 1036]
 [1229 1931]]
정확도: 0.6719, 정밀도: 0.6508, 재현율: 0.6111    F1: 0.6303


C:\Users\jeon\anaconda3\envs\ssu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


교차검증 모델 만들기

In [174]:
#교차검증
scores_dt = cross_val_score(dt_clf,vote_data,vote_result,cv=5)
#정확도
accuracy=accuracy_score(y_test,dt_pred)
#정밀도
precision=precision_score(y_test,dt_pred,average='macro')
#재현율
recall=recall_score(y_test,dt_pred,average='macro')
#F1 스코어
f1score=f1_score(y_test,dt_pred,average='macro')

for iter_count,accuracy in enumerate(scores_dt):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,accuracy))
print("DecisionTree 교차 검증시 평균 정확도 : %.4f "%np.mean(scores_dt))   

for iter_count,precision in enumerate(scores_dt):
    print("교차 검증 {0} 정밀도 : {1:f} ".format(iter_count,precision))
print("DecisionTree 교차 검증시 평균 정밀도 : %.4f "%np.mean(scores_dt))   

for iter_count,recall in enumerate(scores_dt):
    print("교차 검증 {0} 재현율 : {1:f} ".format(iter_count,recall))
print("DecisionTree 교차 검증시 평균 재현율 : %.4f "%np.mean(scores_dt))   

for iter_count,f1score in enumerate(scores_dt):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,f1score))
print("DecisionTree 교차 검증시 평균 F1 스코어 : %.4f "%np.mean(scores_dt))   


교차 검증 0 정확도 : 0.615240 
교차 검증 1 정확도 : 0.618282 
교차 검증 2 정확도 : 0.608286 
교차 검증 3 정확도 : 0.606403 
교차 검증 4 정확도 : 0.608229 
DecisionTree 교차 검증시 평균 정확도 : 0.6113 
교차 검증 0 정밀도 : 0.615240 
교차 검증 1 정밀도 : 0.618282 
교차 검증 2 정밀도 : 0.608286 
교차 검증 3 정밀도 : 0.606403 
교차 검증 4 정밀도 : 0.608229 
DecisionTree 교차 검증시 평균 정밀도 : 0.6113 
교차 검증 0 재현율 : 0.615240 
교차 검증 1 재현율 : 0.618282 
교차 검증 2 재현율 : 0.608286 
교차 검증 3 재현율 : 0.606403 
교차 검증 4 재현율 : 0.608229 
DecisionTree 교차 검증시 평균 재현율 : 0.6113 
교차 검증 0 정확도 : 0.615240 
교차 검증 1 정확도 : 0.618282 
교차 검증 2 정확도 : 0.608286 
교차 검증 3 정확도 : 0.606403 
교차 검증 4 정확도 : 0.608229 
DecisionTree 교차 검증시 평균 F1 스코어 : 0.6113 


In [176]:
#Randomforest 교차검증 모델 생성 및 평가
scores_rf = cross_val_score(rf_clf,vote_data,vote_result,cv=5)
#정확도
accuracy_rf=accuracy_score(y_test,rf_pred)
#정밀도
precision_rf=precision_score(y_test,rf_pred,average='macro')
#재현율
recall_rf=recall_score(y_test,rf_pred,average='macro')
#F1 스코어
f1score_rf=f1_score(y_test,rf_pred,average='macro')


for iter_count,accuracy in enumerate(scores_rf):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,accuracy_rf))
print("Randomforest 교차 검증시 평균 정확도 : %f "%np.mean(scores_rf))   

for iter_count,precision in enumerate(scores_rf):
    print("교차 검증 {0} 정밀도 : {1:f} ".format(iter_count,precision_rf))
print("Randomforest 교차 검증시 평균 정밀도 : %f "%np.mean(scores_rf))   

for iter_count,recall in enumerate(scores_rf):
    print("교차 검증 {0} 재현율 : {1:f} ".format(iter_count,recall_rf))
print("Randomforest 교차 검증시 평균 재현율 : %f "%np.mean(scores_rf))   

for iter_count,f1score_rf in enumerate(scores_rf):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,f1score_rf))
print("Randomforest 교차 검증시 평균 F1 스코어 : %f "%np.mean(scores_rf))   



교차 검증 0 정확도 : 0.676373 
교차 검증 1 정확도 : 0.676373 
교차 검증 2 정확도 : 0.676373 
교차 검증 3 정확도 : 0.676373 
교차 검증 4 정확도 : 0.676373 
Randomforest 교차 검증시 평균 정확도 : 0.678420 
교차 검증 0 정밀도 : 0.676681 
교차 검증 1 정밀도 : 0.676681 
교차 검증 2 정밀도 : 0.676681 
교차 검증 3 정밀도 : 0.676681 
교차 검증 4 정밀도 : 0.676681 
Randomforest 교차 검증시 평균 정밀도 : 0.678420 
교차 검증 0 재현율 : 0.677942 
교차 검증 1 재현율 : 0.677942 
교차 검증 2 재현율 : 0.677942 
교차 검증 3 재현율 : 0.677942 
교차 검증 4 재현율 : 0.677942 
Randomforest 교차 검증시 평균 재현율 : 0.678420 
교차 검증 0 정확도 : 0.675793 
교차 검증 1 정확도 : 0.686368 
교차 검증 2 정확도 : 0.675214 
교차 검증 3 정확도 : 0.673041 
교차 검증 4 정확도 : 0.681686 
Randomforest 교차 검증시 평균 F1 스코어 : 0.678420 


아직까지도 전처리의 문제가 있다. 전처리를 좀 더 시간쓰자.

In [194]:
vote_df

,QaA,QaE,QbA,QbE,QcA,QcE,QdA,QdE,QeA,QeE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,0.00,0.000124,1.00,0.000322,1.00,0.000383,0.00,0.000440,0.00,0.000400,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
1,0.00,0.000211,1.00,0.000473,1.00,0.002294,0.00,0.000690,0.00,0.001705,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2,1.00,0.000242,0.00,0.000224,0.25,0.000687,0.00,0.001164,0.50,0.000809,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.75,0.000065,0.00,0.000911,0.00,0.002214,0.75,0.002965,0.00,0.000724,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.75,0.000217,1.00,0.000512,1.00,0.000523,0.50,0.000366,0.00,0.000657,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36420,0.00,0.000119,1.00,0.000291,1.00,0.000544,0.00,0.000748,0.00,0.000230,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36421,0.50,0.000186,0.75,0.000312,1.00,0.000662,0.25,0.000436,0.00,0.000423,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36422,0.25,0.000245,0.75,0.000383,0.25,0.000785,0.00,0.000668,0.25,0.000951,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
36423,1.00,0.000111,0.00,0.000297,0.00,0.000550,0.50,0.000378,1.00,0.000294,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [199]:
droplist=['married']
vote_droplist=vote_df.drop(droplist,axis=1,inplace=True)
zero_features=['engnat','education','urban']
total_count=vote_df['engnat'].count()
for feature in zero_features:
    zero_count=vote_df[vote_df[feature]==0][feature].count()
    print('{0}건수는 {1}, 퍼센트는{2:.2f}%'.format(feature,zero_count,
                                            100*zero_count/total_count))
vote_df[zero_features]=vote_df[zero_features].replace(0,
                                            vote_df[zero_features].mean())

KeyError: "['married'] not found in axis"

In [200]:
myindexlist=[]
for i in range(0,72):
    myindexlist.append(i)
del myindexlist[57]
myindexlist

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71]

In [201]:
vote_result=vote_df['voted']
vote_data=vote_df.iloc[:,myindexlist]
vote_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34514 entries, 0 to 36424
Data columns (total 71 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   QaA        34514 non-null  float64
 1   QaE        34514 non-null  float64
 2   QbA        34514 non-null  float64
 3   QbE        34514 non-null  float64
 4   QcA        34514 non-null  float64
 5   QcE        34514 non-null  float64
 6   QdA        34514 non-null  float64
 7   QdE        34514 non-null  float64
 8   QeA        34514 non-null  float64
 9   QeE        34514 non-null  float64
 10  QfA        34514 non-null  float64
 11  QfE        34514 non-null  float64
 12  QgA        34514 non-null  float64
 13  QgE        34514 non-null  float64
 14  QhA        34514 non-null  float64
 15  QhE        34514 non-null  float64
 16  QiA        34514 non-null  float64
 17  QiE        34514 non-null  float64
 18  QjA        34514 non-null  float64
 19  QjE        34514 non-null  float64
 20  QkA   

In [202]:
X_train,X_test,y_train,y_test=train_test_split(vote_data, vote_result, test_size=0.2, random_state=0)
print("트레인 데이터 분포 : \n",y_train.value_counts())
print("\n테스트 데이터 분포 : \n",y_test.value_counts())



트레인 데이터 분포 : 
 0.0    14948
1.0    12663
Name: voted, dtype: int64

테스트 데이터 분포 : 
 0.0    3743
1.0    3160
Name: voted, dtype: int64


In [204]:
dt_clf=DecisionTreeClassifier(random_state=0)
rf_clf=RandomForestClassifier(random_state=0)
lr_clf=LogisticRegression()

dt_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
lr_clf.fit(X_train,y_train)

dt_pred=dt_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
lr_pred=lr_clf.predict(X_test)





print("결정 트리")
get_clf_eval(y_test,dt_pred)
print("랜덤 포레스트")
get_clf_eval(y_test,rf_pred)
print("로지스틱")
get_clf_eval(y_test,lr_pred)

#교차검증
scores_dt = cross_val_score(dt_clf,vote_data,vote_result,cv=5)
#정확도
accuracy=accuracy_score(y_test,dt_pred)
#정밀도
precision=precision_score(y_test,dt_pred,average='macro')
#재현율
recall=recall_score(y_test,dt_pred,average='macro')
#F1 스코어
f1score=f1_score(y_test,dt_pred,average='macro')

for iter_count,accuracy in enumerate(scores_dt):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,accuracy))
print("DecisionTree 교차 검증시 평균 정확도 : %.4f "%np.mean(scores_dt))   

for iter_count,precision in enumerate(scores_dt):
    print("교차 검증 {0} 정밀도 : {1:f} ".format(iter_count,precision))
print("DecisionTree 교차 검증시 평균 정밀도 : %.4f "%np.mean(scores_dt))   

for iter_count,recall in enumerate(scores_dt):
    print("교차 검증 {0} 재현율 : {1:f} ".format(iter_count,recall))
print("DecisionTree 교차 검증시 평균 재현율 : %.4f "%np.mean(scores_dt))   

for iter_count,f1score in enumerate(scores_dt):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,f1score))
print("DecisionTree 교차 검증시 평균 F1 스코어 : %.4f "%np.mean(scores_dt))   

#Randomforest 교차검증 모델 생성 및 평가
scores_rf = cross_val_score(rf_clf,vote_data,vote_result,cv=5)
#정확도
accuracy_rf=accuracy_score(y_test,rf_pred)
#정밀도
precision_rf=precision_score(y_test,rf_pred,average='macro')
#재현율
recall_rf=recall_score(y_test,rf_pred,average='macro')
#F1 스코어
f1score_rf=f1_score(y_test,rf_pred,average='macro')


for iter_count,accuracy in enumerate(scores_rf):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,accuracy_rf))
print("Randomforest 교차 검증시 평균 정확도 : %f "%np.mean(scores_rf))   

for iter_count,precision in enumerate(scores_rf):
    print("교차 검증 {0} 정밀도 : {1:f} ".format(iter_count,precision_rf))
print("Randomforest 교차 검증시 평균 정밀도 : %f "%np.mean(scores_rf))   

for iter_count,recall in enumerate(scores_rf):
    print("교차 검증 {0} 재현율 : {1:f} ".format(iter_count,recall_rf))
print("Randomforest 교차 검증시 평균 재현율 : %f "%np.mean(scores_rf))   

for iter_count,f1score_rf in enumerate(scores_rf):
    print("교차 검증 {0} 정확도 : {1:f} ".format(iter_count,f1score_rf))
print("Randomforest 교차 검증시 평균 F1 스코어 : %f "%np.mean(scores_rf))   





C:\Users\jeon\anaconda3\envs\ssu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


결정 트리
오차 행렬
[[2415 1328]
 [1371 1789]]
정확도: 0.6090, 정밀도: 0.5739, 재현율: 0.5661    F1: 0.5700
랜덤 포레스트
오차 행렬
[[2453 1290]
 [ 946 2214]]
정확도: 0.6761, 정밀도: 0.6318, 재현율: 0.7006    F1: 0.6645
로지스틱
오차 행렬
[[2690 1053]
 [1207 1953]]
정확도: 0.6726, 정밀도: 0.6497, 재현율: 0.6180    F1: 0.6335
교차 검증 0 정확도 : 0.603795 
교차 검증 1 정확도 : 0.610169 
교차 검증 2 정확도 : 0.609011 
교차 검증 3 정확도 : 0.594379 
교차 검증 4 정확도 : 0.598522 
DecisionTree 교차 검증시 평균 정확도 : 0.6032 
교차 검증 0 정밀도 : 0.603795 
교차 검증 1 정밀도 : 0.610169 
교차 검증 2 정밀도 : 0.609011 
교차 검증 3 정밀도 : 0.594379 
교차 검증 4 정밀도 : 0.598522 
DecisionTree 교차 검증시 평균 정밀도 : 0.6032 
교차 검증 0 재현율 : 0.603795 
교차 검증 1 재현율 : 0.610169 
교차 검증 2 재현율 : 0.609011 
교차 검증 3 재현율 : 0.594379 
교차 검증 4 재현율 : 0.598522 
DecisionTree 교차 검증시 평균 재현율 : 0.6032 
교차 검증 0 정확도 : 0.603795 
교차 검증 1 정확도 : 0.610169 
교차 검증 2 정확도 : 0.609011 
교차 검증 3 정확도 : 0.594379 
교차 검증 4 정확도 : 0.598522 
DecisionTree 교차 검증시 평균 F1 스코어 : 0.6032 
교차 검증 0 정확도 : 0.676083 
교차 검증 1 정확도 : 0.676083 
교차 검증 2 정확도 : 0.676083 
교차 검증 3 정확도 : 0.676083 

현재 데이터 전처리 상황:
    결혼 여부,hand 지우고 각 데이터들의 무응답 값을 평균값으로 대체 후에 학습 돌려봤음.약간의 성능 향상은 있으나 여전히 평가 성능이 낮음.